In [1]:
#Importing all the required libraries
import numpy as np
import pandas as pd #For loading and handling dataset
import re
from string import punctuation
import nltk #for nlp
from nltk.corpus import stopwords # for the collection of stopping words
nltk.download('stopwords')
from sklearn.model_selection import train_test_split #for splitting the data into training and testing
from tensorflow import keras
from keras.preprocessing.text import Tokenizer #To encode the text into integer array
from keras.preprocessing.sequence import pad_sequences #Helps in padding and truncating the sequence
import matplotlib.pyplot as plt #For plotting graphs
from keras.models import Sequential, load_model #We are using sequential model and we'll also load(call) the saved model
from keras.layers import Dense, LSTM, Embedding, Dropout #Layers in RNN architecture
from keras.callbacks import ModelCheckpoint #Helps to save the model
from string import punctuation
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [31]:
dataset = pd.read_csv('/content/IMDB Dataset.csv')
print(dataset.tail())#Previews the data with first five rows
dataset.describe() #Shows the statistical analysis

                                                  review sentiment
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative


,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [32]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dataset['sentiment'] = le.fit_transform(dataset['sentiment'])

In [52]:
stop_words = stopwords.words('english')#creating a list of stop words


def training_samples():
  input_data=dataset['review']
  output_data=dataset['sentiment']
  #pre-processing data
  input_data=input_data.apply(lambda x: x.lower())#making the words lowercase
  input_data = input_data.apply(lambda x:''.join([c for c in x if c not in punctuation]))#removing characters
  input_data=input_data.apply(lambda x : [i for i in x.split() if i not in stop_words]) #removing stopwords

  return input_data, output_data



input_data, output_data = training_samples()


#Finding the average of words in review
length = [len(i) for i in input_data]
max_length=np.mean(length)


In [57]:
type(input_data)

pandas.core.series.Series

In [34]:
print(dataset['review'])
print(input_data)

0        One of the other reviewers has mentioned that ...
1        A wonderful little production. <br /><br />The...
2        I thought this was a wonderful way to spend ti...
3        Basically there's a family where a little boy ...
4        Petter Mattei's "Love in the Time of Money" is...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot, bad dialogue, bad acting, idiotic di...
49997    I am a Catholic taught in parochial elementary...
49998    I'm going to have to disagree with the previou...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object
0        [one, reviewers, mentioned, watching, 1, oz, e...
1        [wonderful, little, production, br, br, filmin...
2        [thought, wonderful, way, spend, time, hot, su...
3        [basically, theres, family, little, boy, jake,...
4        [petter, matteis, love, time, money, visually,...
             

In [21]:
#Exploring the data
#Finding number of unique outputs
Classes = np.unique(dataset['sentiment'])
#Finding the maximum number of unique words
Max_num_words = len(np.unique(dataset['review']))

#printing observations
print('Classes: ',Classes)
print('Maximum number of unique words: ',Max_num_words)

Classes:  [0 1]
Maximum number of unique words:  49582


In [22]:
tokens= Tokenizer(lower=False)# Since the data is converted to lowercase before
tokens.fit_on_texts(input_data)
input_data = tokens.texts_to_sequences(input_data)

input_data = pad_sequences(input_data,maxlen=127,padding='post',truncating='post')

train_input,test_input,train_output,test_output=train_test_split(input_data,output_data,test_size=0.2)#test_input_size=0.2*input_data

total_words=len(tokens.word_index) + 1 #word_index0 is reserved to distinguish between pad and unknown

In [23]:
embed=32 # dimensions of embeddding
LSTM_SIZE=64 #number of hidden layers
model = Sequential()
model.add(Embedding(total_words,embed,input_length=127))
model.add(LSTM(LSTM_SIZE))
model.add(Dense(1,activation='sigmoid'))#activation is sigmoid as output is either 0 or 1
model.compile(optimizer='adam',loss='binary_crossentropy',metrics =['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 127, 32)           5809408   
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 5834305 (22.26 MB)
Trainable params: 5834305 (22.26 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [24]:
#Using mini-batch learning method with batch_size 200 and 5 epochs
#Adding a callback called checkpoint which saves the model if accuracy is increased from previous epoch
checkpoint=ModelCheckpoint('sentiment/LSTM.h5',monitor='accuracy',save_best_only=True,verbose=2)
model.fit(train_input,train_output,batch_size=200,epochs=5,callbacks=[checkpoint])

Epoch 1/5
200/200 [==============================] - ETA: 0s - loss: 0.6743 - accuracy: 0.5600
Epoch 1: accuracy improved from -inf to 0.55998, saving model to sentiment/LSTM.h5
200/200 [==============================] - 70s 336ms/step - loss: 0.6743 - accuracy: 0.5600
Epoch 2/5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


200/200 [==============================] - ETA: 0s - loss: 0.5450 - accuracy: 0.7069
Epoch 2: accuracy improved from 0.55998 to 0.70692, saving model to sentiment/LSTM.h5
200/200 [==============================] - 64s 320ms/step - loss: 0.5450 - accuracy: 0.7069
Epoch 3/5
200/200 [==============================] - ETA: 0s - loss: 0.2357 - accuracy: 0.9132
Epoch 3: accuracy improved from 0.70692 to 0.91322, saving model to sentiment/LSTM.h5
200/200 [==============================] - 64s 321ms/step - loss: 0.2357 - accuracy: 0.9132
Epoch 4/5
200/200 [==============================] - ETA: 0s - loss: 0.1130 - accuracy: 0.9650
Epoch 4: accuracy improved from 0.91322 to 0.96497, saving model to sentiment/LSTM.h5
200/200 [==============================] - 64s 318ms/step - loss: 0.1130 - accuracy: 0.9650
Epoch 5/5
200/200 [==============================] - ETA: 0s - loss: 0.0586 - accuracy: 0.9855
Epoch 5: accuracy improved from 0.96497 to 0.98548, saving model to sentiment/LSTM.h5
200/200 [=

In [25]:
pred = model.predict(test_input)#predicting the labels
true=0
correct=0
ptrue=0
for i,y in enumerate(test_output):
  if pred[i]>0.5:#classifying predicted label as positive if the confidence value is greater than 0.5 and negative otherwise
    ptrue+=1
  if y==1:
    true+=1
  if ((pred[i]>0.5 and  y==1) or (pred[i]<0.5 and y==0)):
    correct+=1

print('Number of positive sentiment predictions:',ptrue)
print('Real positive sentiment : ',true)
print('Number of negative sentiment predictions:',len(test_input)-ptrue)
print('Real negative sentiment : ',len(test_input)-true)
print('Accuracy of the model is :',(correct/len(test_input))*100)

313/313 [==============================] - 8s 23ms/step
Number of positive sentiment predictions: 5048
Real positive sentiment :  4938
Number of negative sentiment predictions: 4952
Real negative sentiment :  5062
Accuracy of the model is : 86.58
